In [2]:
import pandas as pd
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from preprocess import *
from train_model import *

# Load the dataset into a pandas dataframe:

You can download this dataset from here: https://www.kaggle.com/nzalake52/new-york-times-articles

To rerun the code in this project save the dataset to a "data/" folder in the directory of this notebook. 

In [3]:
with open('./data/nytimes_news_articles.txt') as f:
    document = f.readlines()

articles = []
current_doc = []

for line in document:
    if "URL: htt" not in line:
        current_doc.append(line)
    if line == "\n":
        if len(current_doc) != 1:
            articles.append(" ".join(current_doc))
        current_doc = []
        
# Let's create a pandas dataframe        
dataset = pd.DataFrame({'text': articles})

# Print top 5
dataset.head(5)

,text
0,WASHINGTON — Stellar pitching kept the Mets af...
1,Mayor Bill de Blasio’s counsel and chief legal...
2,In the early morning hours of Labor Day last y...
3,It was the Apple Store in New York City before...
4,OMAHA — The United States Olympic swimming tri...


# Running our initial preprocessing

This may take a while.

In [4]:
dataset['processed'] = [preprocess(x) for x in dataset['text'].values]

# Selecting vocabulary size
Remember that larger vocabularies result in a longer training process.

In [5]:
# Let's see our current processed texts.
dataset.head(5)

,text,processed
0,WASHINGTON — Stellar pitching kept the Mets af...,"[washington, stellar, pitch, kept, met, afloat..."
1,Mayor Bill de Blasio’s counsel and chief legal...,"[mayor, bill, de, blasio, counsel, chief, lega..."
2,In the early morning hours of Labor Day last y...,"[earli, morn, hour, labor, day, last, year, gr..."
3,It was the Apple Store in New York City before...,"[appl, store, new, york, citi, thing, appl, st..."
4,OMAHA — The United States Olympic swimming tri...,"[omaha, unit, state, olymp, swim, trial, spect..."


We need to reduce the vocabulary used in our model to a computationally efficient size so let's study the words and find a good vocabulary size.

In [6]:
# Let's choose a top k words to select from:
k = 30000
top_k_words = select_top_k(dataset, k)

Least common words of top 30000
[('trough', 4), ('coomb', 4), ('edmundscom', 4), ('cancercaus', 4), ('bivalv', 4), ('115th', 4), ('shaqiri', 4), ('poprock', 4), ('astley', 4), ('dorki', 4)]


Once you find a value you like, do the final processing:

In [7]:
dataset['processed'] = keep_top_k_words(dataset, k)

# Training our Model

Our train_lda function takes a pandas dataframe with the processed texts in a 'processed' column, a number of topics to train, and a model name to save model files in a model folder for later use.

In [9]:
# import warnings; warnings.simplefilter('ignore') # Ignoring divide by 0 in log runtime warning during training.

In [10]:
import gensim
from gensim import corpora
dictionary = gensim.corpora.Dictionary.load(r'./models/15topics/15topics.dict')
corpus = gensim.corpora.MmCorpus(r'./models/15topics/15topics.mm')
lda = gensim.models.ldamodel.LdaModel.load(r'./models/15topics/15topics.model')

In [10]:
dictionary,corpus,lda = train_lda(dataset, 50, '50topics')

Training model...
Time to train LDA model on  8897 articles:  0.6633241494496663 min
Saving model files...
Model saved in models folder.


# Now let's visualise!

NOTE: The pyLDAvis tool does not show up on the notebook when saved and uploaded so for the purpose of visualisation I have included the images in markdown cells. When running this notebook however you will be able to interact with the tool you see below.

In [ ]:
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

![Screen-shot](./images/50topics.png)

As you can see there are multiple overlapping topics, suggesting perhaps a lower topic number could be used for a general topic model. Let's try this:

In [12]:
dictionary,corpus,lda = train_lda(dataset, 30, '30topics')

Training model...
Time to train LDA model on  8897 articles:  0.5853211323420207 min
Saving model files...
Model saved in models folder.


In [13]:
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
27    -0.143567  0.025392       1        1  11.360741
4     -0.149861  0.136795       2        1   7.808270
22     0.018304  0.090681       3        1   6.126015
1     -0.101219  0.125952       4        1   5.388998
10     0.073815 -0.030314       5        1   5.369947
13     0.089222 -0.038960       6        1   5.179941
6      0.092378  0.051438       7        1   4.549654
5     -0.154134 -0.193552       8        1   4.549108
7      0.156799  0.053005       9        1   3.791584
15     0.014587  0.025424      10        1   3.720631
12     0.145488  0.016565      11        1   3.509800
23     0.010394  0.077471      12        1   3.420524
2     -0.100457  0.127182      13        1   3.252843
26     0.138596 -0.036503      14        1   3.120656
17    -0.002584  0.107582      15        1   2.756171
29    -0.101830  0.102783      16        1   2.637245
0      0.180786 -0.018228      17        1   2.445925
21    -0.138434 -0.026658      18        1   2.414761
20     0.005753  0.029073      19        1   2.382536
11     0.092030 -0.016911      20        1   2.195532
25    -0.150842 -0.191219      21        1   2.104280
3     -0.165071  0.153540      22        1   2.043848
19    -0.009287 -0.014328      23        1   1.709435
16    -0.082479 -0.034663      24        1   1.704316
28     0.071120  0.058999      25        1   1.658693
9      0.173576 -0.047490      26        1   1.366140
24     0.134013 -0.084298      27        1   0.970577
18    -0.129347 -0.196355      28        1   0.968893
14    -0.110424 -0.171897      29        1   0.829626
8      0.142673 -0.080506      30        1   0.663308, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
390   Default  48661.000000          mr  48661.000000  30.0000  30.0000
194   Default  61435.000000        said  61435.000000  29.0000  29.0000
676   Default  10802.000000     compani  10802.000000  28.0000  28.0000
92    Default   8055.000000        game   8055.000000  27.0000  27.0000
2491  Default   6504.000000       trump   6504.000000  26.0000  26.0000
436   Default  15075.000000       state  15075.000000  25.0000  25.0000
1136  Default   7094.000000     million   7094.000000  24.0000  24.0000
1620  Default   7091.000000     percent   7091.000000  23.0000  23.0000
391   Default  12256.000000          ms  12256.000000  22.0000  22.0000
429   Default   5143.000000      school   5143.000000  21.0000  21.0000
171   Default   7607.000000        play   7607.000000  20.0000  20.0000
1217  Default   4923.000000       court   4923.000000  19.0000  19.0000
1367  Default   4631.000000       women   4631.000000  18.0000  18.0000
201   Default   5179.000000      season   5179.000000  17.0000  17.0000
4114  Default   2856.000000       china   2856.000000  16.0000  16.0000
553   Default   5995.000000        hous   5995.000000  15.0000  15.0000
570   Default   5054.000000         law   5054.000000  14.0000  14.0000
403   Default   4632.000000       polic   4632.000000  13.0000  13.0000
1642  Default   3745.000000       senat   3745.000000  12.0000  12.0000
626   Default   7414.000000        show   7414.000000  11.0000  11.0000
392   Default  19229.000000         new  19229.000000  10.0000  10.0000
2184  Default   4196.000000  republican   4196.000000   9.0000   9.0000
172   Default   4074.000000      player   4074.000000   8.0000   8.0000
1814  Default   3127.000000    european   3127.000000   7.0000   7.0000
539   Default   6965.000000      govern   6965.000000   6.0000   6.0000
1668  Default   3770.000000        vote   3770.000000   5.0000   5.0000
2366  Default   3377.000000     student   3377.000000   4.0000   4.0000
2712  Default   3025.000000       music   3025.000000   3.0000   3.0000
2657  Default   3233.000000         art   3233.000000   2.0000   2.00

![Screen-shot](./images/30topics.png)

Let's decrease our topics one more time..

In [14]:
dictionary,corpus,lda = train_lda(dataset, 15, '15topics')

Training model...
Time to train LDA model on  8897 articles:  0.3784134825070699 min
Saving model files...
Model saved in models folder.


In [15]:
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.117476  0.072711       1        1  20.902615
12     0.026240  0.066503       2        1   9.381370
10    -0.092827  0.026067       3        1   9.178273
2      0.076969  0.145259       4        1   9.043634
3     -0.077032  0.018015       5        1   6.961618
4     -0.058432 -0.018992       6        1   6.631690
6     -0.042434 -0.031150       7        1   6.156432
14     0.210137 -0.134370       8        1   6.131354
8      0.180353  0.138197       9        1   5.745621
9     -0.119982 -0.026732      10        1   5.030302
1      0.003781  0.129959      11        1   4.889521
13    -0.158911 -0.051640      12        1   3.776541
11     0.166477 -0.167240      13        1   3.673402
5     -0.032505 -0.204102      14        1   1.361981
7     -0.199308  0.037515      15        1   1.135652, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
390   Default  49715.000000          mr  49715.000000  30.0000  30.0000
194   Default  61145.000000        said  61145.000000  29.0000  29.0000
2491  Default   6559.000000       trump   6559.000000  28.0000  28.0000
676   Default  10654.000000     compani  10654.000000  27.0000  27.0000
92    Default   8139.000000        game   8139.000000  26.0000  26.0000
1620  Default   7275.000000     percent   7275.000000  25.0000  25.0000
539   Default   7030.000000      govern   7030.000000  24.0000  24.0000
436   Default  15093.000000       state  15093.000000  23.0000  23.0000
4114  Default   2756.000000       china   2756.000000  22.0000  22.0000
1367  Default   4469.000000       women   4469.000000  21.0000  21.0000
171   Default   7767.000000        play   7767.000000  20.0000  20.0000
172   Default   4406.000000      player   4406.000000  19.0000  19.0000
2184  Default   4002.000000  republican   4002.000000  18.0000  18.0000
1108  Default   5935.000000     countri   5935.000000  17.0000  17.0000
403   Default   4941.000000       polic   4941.000000  16.0000  16.0000
491   Default   3871.000000     clinton   3871.000000  15.0000  15.0000
232   Default   5858.000000        team   5858.000000  14.0000  14.0000
201   Default   5316.000000      season   5316.000000  13.0000  13.0000
1217  Default   5197.000000       court   5197.000000  12.0000  12.0000
305   Default   4802.000000    campaign   4802.000000  11.0000  11.0000
1019  Default   6127.000000       world   6127.000000  10.0000  10.0000
570   Default   5023.000000         law   5023.000000   9.0000   9.0000
1814  Default   2896.000000    european   2896.000000   8.0000   8.0000
596   Default   4830.000000       parti   4830.000000   7.0000   7.0000
2804  Default   2862.000000      island   2862.000000   6.0000   6.0000
1642  Default   3392.000000       senat   3392.000000   5.0000   5.0000
587   Default   5958.000000      offici   5958.000000   4.0000   4.0000
1136  Default   6770.000000     million   6770.000000   3.0000   3.0000
1013  Default   7623.000000        unit   7623.000000   2.0000   2.0000
2712  Default   3183.000000       music   3183.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
4876  Topic15    139.013275    mainland    146.244934   4.4272  -5.7935
2987  Topic15    630.488403      climat    710.014038   4.3592  -4.2816
4982  Topic15    208.609024     christi    225.461517   4.4003  -5.3876
2988  Topic15    242.265335        coal    270.812622   4.3666  -5.2380
4857  Topic15    249.326843   communist    318.144470   4.2342  -5.2093
838   Topic15    329.187988   australia    553.801697   3.9578  -4.9314
6150  Topic15    216.245987      carbon    323.273010   4.0759  -5.3517
7384  Topic15    270.773132      disney    464.343048   3.9386  -5.1268
1376  Topic15    491.223297        anim   1113.801880   3.6593  -4.5312
839   Topic15  

![Screen-shot](./images/15topics.png)

# Comments:

As you can see from this simple example, the iterative process of finding a good general topic model becomes slightly more intuitive when visualising the topic distribution. Studying each topic and their top words can also give insights as to how to change the vocabulary for a better model and how to change the parameters in our train_lda function that define the representation of topics. 

The LDA model is an extremely useful tool for any topic modeling task, and it has a variety of functions including finding similarity between documents (for this you may add "from gensim import similarities" in the imports). For example, this can be used to generate a simple recommendation system for documents in any system.